# Building Safe and Persistent AI Agents with Guardrails and Sessions

Hey everyone! Welcome back to another exciting lesson on Agentic AI! 🎉

In this lab, we'll explore:

- **Input Guardrails**: Validating and filtering user inputs before they reach your agents
- **Output Guardrails**: Ensuring agent responses meet quality and safety standards
- **Guardrail Functions**: Creating custom validation logic with tripwire patterns
- **SQLite Sessions**: Managing conversation state with in-memory and persistent storage
- **Session Management**: Building agents that remember context across multiple interactions

Let's get started!

### Input Guardrails and Guardrail Functions

In [1]:
import os
from agents import Agent, Runner, InputGuardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, input_guardrail
from dotenv import load_dotenv
from pydantic import BaseModel, Field
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [5]:
refund_agent = Agent(
    name="RefundAgent",
    model="gpt-4.1-mini",
    instructions=
    "You are a refund agent that processes refund requests based on company policy."\
    "For now say there are no refunds available",
    handoff_description="Handles refund requests from customers."
)

retention_agent = Agent(
    name="RetentionAgent",
    model="gpt-4.1-mini",
    instructions=
    "You are a retention agent that offers retention deals to customers considering " \
    "cancellation. For now say that we are offering 20% off on everything",
    handoff_description="Handles customer retention and offers deals to prevent cancellations."
)

upsell_agent = Agent(
    name="SalesAgent",
    model="gpt-4.1-mini",
    instructions=
    "You are a sales agent that offers upsell options to customers based on their " \
    "current subscriptions. For now say we have a premium plan available at 30% off.",
    handoff_description="Handles upselling and cross-selling to customers."
)

customer_service_agent = Agent(
    name="CustomerServiceTriageAgent",
    model="gpt-4.1-mini",
    instructions=
    "You are a customer service triage agent that chats with the user " \
    "about their query then directs customer inquiries to the appropriate department " \
    "based on the issue described.",
    handoffs=[refund_agent, retention_agent, upsell_agent],
    input_guardrails=[guardrail_tool]
)

<div style="border-radius:16px;background:#2e3440;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#88c0d0;font-size:1.25em">Info:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li><code>input_correct</code> is now a <b>boolean</b> instead of a string ("yes"/"no") for cleaner validation logic.</li>
    <li>We use <code>not final_output.input_correct</code> in the <code>tripwire_triggered</code> parameter because:
      <ul style="margin-top:0.3em">
        <li>When input is <b>correct</b>: <code>input_correct = True</code> → <code>tripwire_triggered = False</code> (no blocking)</li>
        <li>When input is <b>incorrect</b>: <code>input_correct = False</code> → <code>tripwire_triggered = True</code> (blocks execution)</li>
      </ul>
    </li>
    <li>The <code>not</code> keyword inverts the boolean to match the expected tripwire behavior!</li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#88c0d0;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💡</div>
</div>

In [2]:
class InputGuardrailResponse(BaseModel):
    input_correct: bool = Field(..., description="Indicates if the input is appropriate")
    reason: str = Field(..., description="If the input is inappropriate, provide a reason.")

In [3]:
input_guardrail_agent = Agent(
    name="InputGuardrailAgent",
    model="gpt-4.1-mini",
    instructions=
    "Make sure the user input is safe for processing. " \
    "Make sure the inquiry is about either books or clothes and not about math.",
    output_type=InputGuardrailResponse
)

In [4]:
@input_guardrail
async def guardrail_tool(ctx, agent, input_data):
    results = await Runner.run(input_guardrail_agent, input_data, context=ctx.context)
    final_output = results.final_output
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered = not final_output.input_correct
    )

In [ ]:
results = await Runner.run(customer_service_agent, "I want to learn how to hack into my wifi router.")
print(results)

In [ ]:
try:
    results = await Runner.run(customer_service_agent, "I want to learn how to hack into my wifi router.")
    print(results)
except InputGuardrailTripwireTriggered as e:
    print("Input guardrail tripwire triggered: ", e)

### Output Guardrails

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4);word-wrap:break-word;overflow-wrap:break-word">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li><b>Build an Output Guardrail!</b> Now that you've seen input guardrails, create an output guardrail that validates agent responses.</li>
    <li>Your guardrail should:
      <ul style="margin-top:0.3em">
        <li>Create a Pydantic model with <code>output_appropriate</code> and <code>reason</code></li>
        <li>Use <code>@output_guardrail</code> decorator</li>
        <li>Validate quality/safety (no harmful content, stays on topic)</li>
        <li>Return <code>GuardrailFunctionOutput</code> with tripwire status</li>
        <li>Test with an agent</li>
      </ul>
    </li>
    <li><b>Hint:</b> Similar to input guardrails, but validates agent responses!</li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>

[OpenAI Agents SDK Docs for Guardrails](https://openai.github.io/openai-agents-python/guardrails/)

[Context Management](https://openai.github.io/openai-agents-python/context/)

# Session using SQLite

### In-memory Sessions

In [6]:
from agents import SQLiteSession

In [7]:
session = SQLiteSession("conversation_123")

In [8]:
customer_service_agent = Agent(
    name="CustomerServiceTriageAgent",
    model="gpt-4.1-mini",
    instructions=
    "You are a customer service triage agent that chats with the user " \
    "about their query then directs customer inquiries to the appropriate department " \
    "based on the issue described.",
    handoffs=[refund_agent, retention_agent, upsell_agent],
    input_guardrails=[guardrail_tool]
)

In [10]:
results = await Runner.run(refund_agent, "I want a refund", session=session)
print(results)

RunResult:
- Last agent: Agent(name="RefundAgent", ...)
- Final output (str):
    Currently, there are no refunds available according to our company policy. If you need assistance with something else, please let me know!
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


### Persistent Sessions

In [11]:
session = SQLiteSession("conversation_123",db_path="db/conversations.db")

In [12]:
customer_service_agent = Agent(
    name="CustomerServiceTriageAgent",
    model="gpt-4.1-mini",
    instructions=
    "You are a customer service triage agent that chats with the user " \
    "about their query then directs customer inquiries to the appropriate department " \
    "based on the issue described.",
    handoffs=[refund_agent, retention_agent, upsell_agent],
    input_guardrails=[guardrail_tool]
)

In [14]:
results = await Runner.run(customer_service_agent, "What books do you have?", session=session)
print(results)

RunResult:
- Last agent: Agent(name="CustomerServiceTriageAgent", ...)
- Final output (str):
    I can help you with your refund request and provide information about our book selection. To assist you best:
    
    1. For the refund, I'll connect you to the refund team.
    2. For books, could you specify the genre or type of books you are interested in? We have a wide range including fiction, non-fiction, self-help, and more.
    
    Would you like me to proceed with the refund request first?
- 1 new item(s)
- 1 raw response(s)
- 1 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


## 📚 Resources

<div style="border-radius:16px;background:#2e3440;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#88c0d0;font-size:1.25em">Helpful Links:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li><a href="https://openai.github.io/openai-agents-python/guardrails/" style="color:#88c0d0">OpenAI Agents SDK - Guardrails Documentation</a></li>
    <li><a href="https://openai.github.io/openai-agents-python/context/" style="color:#88c0d0">OpenAI Agents SDK - Context Management</a></li>
    <li><a href="https://openai.github.io/openai-agents-python/ref/extensions/memory/sqlalchemy_session/" style="color:#88c0d0">OpenAI Agents SDK - SQLite Sessions</a></li>
    <li><a href="https://community.superdatascience.com" style="color:#88c0d0">SuperDataScience Community</a></li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#88c0d0;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💡</div>
</div>